<a href="https://colab.research.google.com/github/ny-yo/kaggle-CommonLit-Readability-Prize/blob/main/03_commonlit_roberta_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain/

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive #インポート
drive.mount('/content/drive/') #GoogleDriveのマウント

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        200           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        327           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        972           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       6030           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      50411            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge      13189            True  
connectx

In [ ]:
!kaggle competitions download -c commonlitreadabilityprize

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

train = pd.read_csv("/content/train.csv.zip")
test = pd.read_csv("/content/test.csv")
sample= pd.read_csv("/content/sample_submission.csv")

In [ ]:
!pip install transformers

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import torch
import torch.nn as nn
import transformers
import random
from torch.utils.data import DataLoader, Dataset
import warnings
warnings.simplefilter('ignore')

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaModel, RobertaTokenizer, AdamW, RobertaForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from transformers import get_linear_schedule_with_warmup

scaler = torch.cuda.amp.GradScaler() # GPUでの高速化。

In [ ]:
MODEL = 'roberta-base'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
tokenizer = RobertaTokenizer.from_pretrained(MODEL) 

In [ ]:
train_data = pd.read_csv('/content/train.csv.zip')
test_data = pd.read_csv('/content/test.csv')

data = pd.concat([train_data,test_data])
text  = '\n'.join(data.excerpt.tolist())

with open('/content/drive/MyDrive/03-roberta_pretrain_text.txt','w') as f:
    f.write(text)

In [ ]:
model = RobertaForMaskedLM.from_pretrained(MODEL)
tokenizer = RobertaTokenizer.from_pretrained(MODEL)
tokenizer.save_pretrained('/content/drive/MyDrive/03-roberta_pretrain_base')

('/content/drive/MyDrive/03-roberta_pretrain_base/tokenizer_config.json',
 '/content/drive/MyDrive/03-roberta_pretrain_base/special_tokens_map.json',
 '/content/drive/MyDrive/03-roberta_pretrain_base/vocab.json',
 '/content/drive/MyDrive/03-roberta_pretrain_base/merges.txt',
 '/content/drive/MyDrive/03-roberta_pretrain_base/added_tokens.json')

In [ ]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/03-roberta_pretrain_text.txt", #mention train text file here
    block_size=256)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/03-roberta_pretrain_text.txt", #mention valid text file here
    block_size=256)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/03-roberta_pretrain_base_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    evaluation_strategy= 'steps',
    save_total_limit=5,
    eval_steps=150,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

In [ ]:
trainer.train()
trainer.save_model(f'/content/drive/MyDrive/03-roberta_pretrain_base')

***** Running training *****
  Num examples = 7114
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 13350


Step,Training Loss,Validation Loss
150,No log,1.620158
300,No log,1.583752
450,No log,1.587865
600,1.756800,1.560121
750,1.756800,1.545520
900,1.756800,1.519295
1050,1.710900,1.510527
1200,1.710900,1.486736
1350,1.710900,1.476563
1500,1.656000,1.465057


***** Running Evaluation *****
  Num examples = 7114
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/03-roberta_pretrain_base_chk/checkpoint-150
Configuration saved in /content/drive/MyDrive/03-roberta_pretrain_base_chk/checkpoint-150/config.json
Model weights saved in /content/drive/MyDrive/03-roberta_pretrain_base_chk/checkpoint-150/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 7114
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/03-roberta_pretrain_base_chk/checkpoint-300
Configuration saved in /content/drive/MyDrive/03-roberta_pretrain_base_chk/checkpoint-300/config.json
Model weights saved in /content/drive/MyDrive/03-roberta_pretrain_base_chk/checkpoint-300/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 7114
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/03-roberta_pretrain_base_chk/checkpoint-450
Configuration saved in /content/drive/MyDrive/03-roberta_pretrain_base_chk/checkpoint-45

In [ ]:
print("finish")

finish
